In [1]:
import os 
cwd = os.getcwd()
print(cwd)

D:\xhou4\ML_CVR\smooth8\ML_data\script_test


In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import time

import os
import fnmatch
import nibabel as nib
import glob

In [3]:
import sys
sys.path.append('../script_test/')

In [4]:
cwd

'D:\\xhou4\\ML_CVR\\smooth8\\ML_data\\script_test'

In [5]:
import torch.nn as nn
import torch
from net_utils import unetConv2, unetUp, UnetDsv2
import torch.nn.functional as F
from networks_other import init_weights
from grid_attention_layer import GridAttentionBlock2D

class unet_single_att_dsv_2D(nn.Module):

    def __init__(self, feature_scale=8, n_classes=1, is_deconv=True, in_channels=1,
                 nonlocal_mode='concatenation', attention_dsample=(2, 2), is_batchnorm=True):
        super(unet_single_att_dsv_2D, self).__init__()
        self.is_deconv = is_deconv
        self.in_channels = in_channels
        self.is_batchnorm = is_batchnorm
        self.feature_scale = feature_scale

        filters = [64, 128, 256, 512, 1024]
        filters = [int(x / self.feature_scale) for x in filters]
#         self.ssm_conv = nn.Sequential(nn.Conv3d(1, self.in_channels, kernel_size=(5, 3, 3),
#                                                  stride=(1, 1, 1), padding=(0, 1, 1)),
#                                        nn.ReLU(inplace=True),
#                                        nn.Conv3d(self.in_channels, self.in_channels, kernel_size=(5, 3, 3),
#                                                  stride=(1, 1, 1), padding=(0, 1, 1))
#                                        )

        # downsampling
        self.conv1 = unetConv2(self.in_channels, filters[0], self.is_batchnorm)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv2 = unetConv2(filters[0], filters[1], self.is_batchnorm)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv3 = unetConv2(filters[1], filters[2], self.is_batchnorm)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv4 = unetConv2(filters[2], filters[3], self.is_batchnorm)
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2, 2))

        self.center = unetConv2(filters[3], filters[4], self.is_batchnorm)
        self.gating = UnetGridGatingSignal2(filters[4], filters[4], kernel_size=(1, 1), is_batchnorm=self.is_batchnorm)

        # attention blocks
        self.attentionblock2 = MultiAttentionBlock(in_size=filters[1], gate_size=filters[2], inter_size=filters[1],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor=attention_dsample)
        self.attentionblock3 = MultiAttentionBlock(in_size=filters[2], gate_size=filters[3], inter_size=filters[2],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor=attention_dsample)
        self.attentionblock4 = MultiAttentionBlock(in_size=filters[3], gate_size=filters[4], inter_size=filters[3],
                                                   nonlocal_mode=nonlocal_mode, sub_sample_factor=attention_dsample)

        # upsampling
        self.up_concat4 = unetUp(filters[4], filters[3], self.is_deconv)
        self.up_concat3 = unetUp(filters[3], filters[2], self.is_deconv)
        self.up_concat2 = unetUp(filters[2], filters[1], self.is_deconv)
        self.up_concat1 = unetUp(filters[1], filters[0], self.is_deconv)

        # deep supervision
        self.dsv4 = UnetDsv2(in_size=filters[3], out_size=n_classes, scale_factor=8)
        self.dsv3 = UnetDsv2(in_size=filters[2], out_size=n_classes, scale_factor=4)
        self.dsv2 = UnetDsv2(in_size=filters[1], out_size=n_classes, scale_factor=2)
        self.dsv1 = nn.Conv2d(in_channels=filters[0], out_channels=n_classes, kernel_size=1)

        # final conv (without any concat)
        self.final = nn.Conv2d(n_classes*4, n_classes, 1)

        # initialise weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init_weights(m, init_type='kaiming')
            elif isinstance(m, nn.BatchNorm2d):
                init_weights(m, init_type='kaiming')

    def forward(self, inputs):

        # Slice Sequence Modeling Block

#         inputs = inputs.unsqueeze(dim=1)
#         ssm = self.ssm_conv(inputs)
#         inputs = ssm.squeeze()

#         if inputs.dim() == 3:
#             inputs = inputs.unsqueeze(dim=0)

        # Feature Extraction
        conv1 = self.conv1(inputs)
        maxpool1 = self.maxpool1(conv1)

        conv2 = self.conv2(maxpool1)
        maxpool2 = self.maxpool2(conv2)

        conv3 = self.conv3(maxpool2)
        maxpool3 = self.maxpool3(conv3)

        conv4 = self.conv4(maxpool3)
        maxpool4 = self.maxpool4(conv4)

        # Gating Signal Generation
        center = self.center(maxpool4)
        gating = self.gating(center)

        # Attention Mechanism
        # Upscaling Part (Decoder)
        g_conv4, att4 = self.attentionblock4(conv4, gating)
        up4 = self.up_concat4(g_conv4, center)
        g_conv3, att3 = self.attentionblock3(conv3, up4)
        up3 = self.up_concat3(g_conv3, up4)
        g_conv2, att2 = self.attentionblock2(conv2, up3)
        up2 = self.up_concat2(g_conv2, up3)
        up1 = self.up_concat1(conv1, up2)

        # Deep Supervision
        dsv4 = self.dsv4(up4)
        dsv3 = self.dsv3(up3)
        dsv2 = self.dsv2(up2)
        dsv1 = self.dsv1(up1)
        final = self.final(torch.cat([dsv1, dsv2, dsv3, dsv4], dim=1))

        return final

class UnetGridGatingSignal2(nn.Module):
    def __init__(self, in_size, out_size, kernel_size=(1,1), is_batchnorm=True):
        super(UnetGridGatingSignal2, self).__init__()

        if is_batchnorm:
            self.conv1 = nn.Sequential(nn.Conv2d(in_size, out_size, kernel_size, (1,1), (0,0)),
                                       nn.BatchNorm2d(out_size),
                                       nn.ReLU(inplace=True),
                                       )
        else:
            self.conv1 = nn.Sequential(nn.Conv2d(in_size, out_size, kernel_size, (1,1), (0,0)),
                                       nn.ReLU(inplace=True),
                                       )

        # initialise the blocks
        for m in self.children():
            init_weights(m, init_type='kaiming')

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        return outputs


class MultiAttentionBlock(nn.Module):
    def __init__(self, in_size, gate_size, inter_size, nonlocal_mode, sub_sample_factor):
        super(MultiAttentionBlock, self).__init__()
        self.gate_block_1 = GridAttentionBlock2D(in_channels=in_size, gating_channels=gate_size,
                                                 inter_channels=inter_size, mode=nonlocal_mode,
                                                 sub_sample_factor=sub_sample_factor)
        self.combine_gates = nn.Sequential(nn.Conv2d(in_size, in_size, kernel_size=1, stride=1, padding=0),
                                           #nn.BatchNorm2d(in_size),
                                           nn.ReLU(inplace=True)
                                           )

        # initialize the blocks
        for m in self.children():
            if m.__class__.__name__.find('GridAttentionBlock3D') != -1: continue
            init_weights(m, init_type='kaiming')

    def forward(self, input, gating_signal):
        gate_1, attention_1 = self.gate_block_1(input, gating_signal)

        return self.combine_gates(gate_1), attention_1


In [6]:
import torch
from torch.utils import data
import nibabel as nib

class CVRDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, data_root):
        'initialization'
        self.input_IDs = []
        self.label_IDs = []
        'Find the file location'
        for subname in os.listdir(data_root):
            if os.path.isdir(os.path.join(data_root, subname)):
                for subfile in os.listdir(os.path.join(data_root, subname)):
                    if fnmatch.fnmatch(subfile, 'bold_mean_ws_[0]*'):
            
                        sub_filepath = os.path.join(data_root, subname, subfile)
                        self.input_IDs.append(sub_filepath)
                    
                for tarfile in os.listdir(os.path.join(data_root, subname, 'HC')):
                    if fnmatch.fnmatch(tarfile, 'CVR_HC_clean_[0]*'):
                        
                        target_filepath = os.path.join(data_root, subname, 'HC', tarfile)
                        self.label_IDs.append(target_filepath)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.input_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        
        'initialization'
        input_x = []
        label_y = []
        
        # Select input
        input_ID = self.input_IDs[index]
#         print(input_ID)
        
        # Load input
        img = nib.load(input_ID)
        train_image = img.get_fdata()
        train_image = train_image.astype(np.float32)
        img.uncache()
        
#         train_image = np.pad(train_image, ((2, 3), (1, 2), (0, 0)), 'constant', constant_values = 0) #pad zero surrounding the image
        train_image = np.transpose(train_image, (2, 0, 1))
        
        A = torch.Tensor(train_image).type(torch.FloatTensor)
        
        # Select label
        label_ID = self.label_IDs[index]
#         print(label_ID)
        
        # Load label
        taimg = nib.load(label_ID)
        label_image = taimg.get_fdata()
        label_image = label_image.astype(np.float32)
        taimg.uncache()
        
#         label_image = np.pad(label_image, ((2, 3), (1, 2)), 'constant', constant_values = 0) #pad zero surrounding the image

        B = torch.Tensor(label_image).type(torch.FloatTensor)
        return [A, B]

In [7]:
import torch
from torch.utils import data

# Generators
training_set = CVRDataset('d:\\xhou4\\ML_CVR\\smooth8\\ML_data\\train')
CVRdata = data.DataLoader(training_set, batch_size=64, shuffle=True)

x, y = next(iter(CVRdata))
print(x.shape)
print(y.shape)

torch.Size([64, 115, 96, 112])
torch.Size([64, 96, 112])


In [8]:
import math
import torch
from torch.utils import data

# Generators
training_set = CVRDataset('d:\\xhou4\\ML_CVR\\smooth8\\ML_data\\train')
dataloader = data.DataLoader(training_set, batch_size=64, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda: 1' if torch.cuda.is_available() else 'cpu')
model = unet_single_att_dsv_2D(in_channels=115, n_classes=1, feature_scale=1).to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.L1Loss()
epochs = 100

total_step = len(dataloader)
loss_list = []

for t in range(epochs):

    running_loss = 0.0
    batch_loss = 0.0
    step_count = 0.0
    for i, (X, y) in enumerate(dataloader):
        X = X.to(device)  # [N, 283, H, W]
        y = y.to(device)  # [N, H, W] with class indices (0, 1)
        prediction = np.squeeze(model(X))  # [N, H, W]

        loss = criterion(prediction, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        if math.isnan(loss.item()) == False:
            batch_loss += loss.item()
            step_count += 1

        if (i + 1) % 10 == 0:
            running_loss += batch_loss
            print('Epoch number {}, Step [{}/{}], Loss: {:.4f} '
                      .format(t + 1, i + 1, total_step, loss.item()))
            batch_loss = 0.0

    loss_list.append(running_loss/step_count)

## Plotting batch-wise train loss curve:
plt.plot(loss_list, '-o', label = 'train_loss', color = 'blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


C:\Users\Windows\Lulab\hfan11\Anaconda3\envs\py3_xh\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
C:\Users\Windows\Lulab\hfan11\Anaconda3\envs\py3_xh\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
C:\Users\Windows\Lulab\hfan11\Anaconda3\envs\py3_xh\lib\site-packages\torch\nn\functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch number 1, Step [10/148], Loss: 0.1345 
Epoch number 1, Step [20/148], Loss: 0.1055 
Epoch number 1, Step [30/148], Loss: 0.0861 
Epoch number 1, Step [40/148], Loss: 0.0755 
Epoch number 1, Step [50/148], Loss: 0.0688 
Epoch number 1, Step [60/148], Loss: 0.0605 
Epoch number 1, Step [70/148], Loss: 0.0555 
Epoch number 1, Step [80/148], Loss: 0.0555 
Epoch number 1, Step [90/148], Loss: 0.0505 
Epoch number 1, Step [100/148], Loss: 0.0474 


KeyboardInterrupt: 

In [16]:
test_dir = 'd:\\xhou4\\ML_CVR\\smooth8\\ML_data\\test'
os.chdir(test_dir)

model.eval()
with torch.no_grad():
    for subname in glob.glob('*\\'):
        test_result = np.zeros((96, 112, 91))

        tensor_x = []
        tensor_y = []

        sub_filename = os.path.join(test_dir, subname, 'bold_corr_rp_c.nii')
        print(sub_filename)
        img = nib.load(sub_filename)
        img_data = img.get_fdata()
        img.uncache()

        for i in range(80):
            train_image = np.squeeze(img_data[:, :, i, :])
            train_image = np.pad(train_image, ((2, 3), (1, 2), (0, 0)), 'constant') #pad zero surrounding the image
            train_image = np.transpose(train_image, (2, 0, 1))
            tensor_x.append(torch.Tensor(train_image))

        target_filename = os.path.join(test_dir, subname, 'HC/CVR_HC_clean.nii')
        print(target_filename)
        taimg = nib.load(target_filename)
        taimg_data = taimg.get_fdata()    
        taimg.uncache()

        for i in range(80):
            label_image = np.squeeze(taimg_data[:, :, i])
            label_image = np.pad(label_image, ((2, 3), (1, 2)), 'constant') #pad zero surrounding the image
            tensor_y.append(torch.Tensor(label_image))

        tensor_x_stacked = torch.stack(tensor_x)
        tensor_y_stacked = torch.stack(tensor_y)

        test_data = torch.utils.data.TensorDataset(tensor_x_stacked,tensor_y_stacked)
        testloader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=1)

        i = 0
        for X, y in testloader:
            X = X.to(device)
        #         plt.imshow(X[0][1]) 
        #         plt.show()
            outputs = np.squeeze(model(X).cpu().clone().numpy())
            test_result[:, :, i] = outputs
            i += 1

        img_output = nib.Nifti1Image(test_result, taimg.affine)
        img_output.get_data_dtype() == np.dtype(np.int16)
        nib.save(img_output, os.path.join(test_dir, subname, 'ML_result_aUnet_XYZ.nii'))


d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_mr1_ma110116\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_mr1_ma110116\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_ac033017\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_ac033017\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_as061418\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_as061418\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_bm011118\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_bm011118\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_hl042519\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_hl042519\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_se082318\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_se082318\HC/CVR_HC_clean.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_ta011818\bold_corr_rp_c.nii
d:\xhou4\ML_CVR\smooth8\ML_data\test\hlu_xmr_

In [17]:
torch.save(model.state_dict(), 'd:/xhou4/ML_CVR/smooth8/ML_data/AUnet_train_model_norm_XYZ_135sub.pt')

### 